# Demo of the forecast.io wrapper to get past and future weather data

Important: you need to register for an apikey here: https://developer.forecast.io/  Put the key you obtain in the opengrid.cfg file as follows:

    [Forecast.io]
    apikey: your_key

In [ ]:
import os, sys, inspect
import pandas as pd
import charts

In [ ]:
# add the sccript path to opengrid to sys.path
script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))

from opengrid.library.config import Config

## Get Forecast.io API key from config file

In [ ]:
config = Config()

#get Forecast.io API Key
api_key = config.get('Forecast.io', 'apikey')

## Import API wrapper module

In [ ]:
from opengrid.library import forecastwrapper

# Get weather data in daily resolution

### Degree Days

Daily resolution has the option of adding degree days.
By default, heating degree days with a base temperature of 16.5°C are added.

Heating degree days are calculated as follows:

$$heatingDegreeDays = max(0 , baseTemp - 0.6 * T_{today} + 0.3 * T_{today-1} + 0.1 * T_{today-2} )$$

Cooling degree days are calculated in an analog way:

$$coolingDegreeDays = max(0, 0.6 * T_{today} + 0.3 * T_{today-1} + 0.1 * T_{today-2} - baseTemp )$$

Add degree days by setting the flag heatingDegreeDays or coolingDegreeDays and supplying heatingBaseTemps and/or coolingBaseTemps as a list (you can add multiple base temperatures, or just a list of 1 element)

### Daytime Averages

Daily resolution has the option of adding average values during the daytime only (between sunrise and sunset). Implemented are daytimeCloudCover and daytimeTemperature, and are included by default. They can be excluded by setting daytimeCloudCover = False or daytimeTemperature = False.

In [ ]:
start = pd.Timestamp('20150813')
end = pd.Timestamp('20150816')

#### Default Weather Dataframe

In [ ]:
WD = forecastwrapper.Weather_Days(api_key=api_key, location='Ukkel', start=start, end=end)

In [ ]:
WD.df

#### Add some more degree days

In [ ]:
WD2 = forecastwrapper.Weather_Days(api_key=api_key, 
                                   location='Ukkel', 
                                   start=start, 
                                   end=end, 
                                   heatingDegreeDays=True, 
                                   heatingBaseTemps=[15,18], 
                                   coolingDegreeDays=True, 
                                   coolingBaseTemps=[18,24],
                                   )

By default, daily weather data is extended with *daytimeCloudCover* and *daytimeTemperature*.  These are the averaged cloud cover and temperature **between sunrise and sunset**. 

In [ ]:
WD2.df.filter(like='daytime')

# Get weather data in hourly resolution

Location can also be coördinates

In [ ]:
WHBrussel = forecastwrapper.Weather_Hours(api_key=api_key, location=[50.8503396, 4.3517103], 
                                   start=pd.Timestamp('20150916'), end=pd.Timestamp('20150918'))
WHBoutersem = forecastwrapper.Weather_Hours(api_key=api_key, location='Kapelstraat 1, 3370 Boutersem', 
                                   start=pd.Timestamp('20150916'), end=pd.Timestamp('20150918'))

In [ ]:
df_combined = pd.merge(WHBrussel.df, WHBoutersem.df, suffixes=('_Brussel', '_Boutersem'), 
                           left_index=True, right_index=True)

In [ ]:
charts.plot(df_combined.filter(like='cloud'), stock=True, show='inline')

# Deal with different timezones

By default, all dataframes are localized to the timezone of the specified location.
If you want, you can specify the tz argument to get the dataframe localized to a specified timezone.

In [ ]:
start = pd.Timestamp('20150815')
end = pd.Timestamp('20150816')

In [ ]:
WDLondon = forecastwrapper.Weather_Days(api_key=api_key, location='London', start=start, end=end, tz='Asia/Singapore')
WDBrussels = forecastwrapper.Weather_Days(api_key=api_key, location='Brussels', start=start, end=end, tz='Asia/Singapore')

In [ ]:
WDLondon.df

In [ ]:
WDBrussels.df